# Construção da base de notícias ESG a partir da captura do Google RSS

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG
- Trabalho de conclusão de curso - MBA Digital Business USP Esalq

In [1]:
import warnings
import nltk
import numpy as np
import pandas as pd
import re
import unidecode
import datetime as dt
from noticias_google_buscador import busca_noticias_google_news
from noticias_processamento_texto import *
from noticias_google_buscador_esg import *


warnings.filterwarnings('ignore')
arquivo_termos_esg = 'datasets/palavras_chave_esg.xlsx'
base_noticias = 'datasets/base_noticias.xlsx'



[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
aplicar_filtros=False
inclui_apenas_nome_empresa = True

df_atual = pd.read_excel(base_noticias)

hash_noticias_ja_existentes = list(df_atual['hash'])

# processa uma empresa especificada
for empresa in [  'brasil biofuels']:
    print('\n' +  str(dt.datetime.now())  + ' Buscando empresa ' + empresa)
    dfEmpresasListadas = recupera_lista_empresas_B3()
    df1 = busca_noticias_google_esg(empresa, inclui_apenas_nome_empresa = True)
    
    if len(df1) >0:
        # ajustes
        df1['data_publicacao'] = df1['data_publicacao'].dt.date
        df1 = ajusta_nomes_empresas_dataframe(df1)

        # cria hash
        df1['hash'] = df1.apply(lambda row: criar_hash_noticia(texto=row['titulo'], empresa=row['empresa'], titulo=row['titulo'], data=row['data_publicacao']), axis=1)
        print(str(dt.datetime.now())  + ' ... noticias encontradas: ' + str(len(df1)))
    
    if len(df1) >0:
        df1 = df1[~df1['hash'].isin(hash_noticias_ja_existentes)]
        print(str(dt.datetime.now())  + ' ... noticias novas: ' + str(len(df1)))
    
    if len(df1) > 0:
        df2 = recupera_noticias_completas(df1)
        print(df1)
        print(str(dt.datetime.now())  + ' ... textos buscados: ' + str(len(df2)))
            
        if aplicar_filtros:
            df3 = filtra_noticias_nao_relacionadas(df2, empresa)
            if len(df3) >0:
                df4 = filtra_citacao_relevante(df3, empresa, dfEmpresasListadas )
                print(str(dt.datetime.now())  + ' ... novas noticias após filtro citaçoes: ' + str(len(df4)))
                df5 = classifica_textos_coletados(df4)
        else:
            df5 = df2
        
        if len(df5) >0:
            df6 = df5
            print(str(dt.datetime.now())  + ' ... novas noticias após filtros: ' + str(len(df6)))
            df_atual = pd.read_excel(base_noticias)

            # corte de data - se necessario inibir noticias alem de uma data
            #df6 = df6[df6.data_publicacao.dt.date < dt.date(2023, 3, 1)]

            df = pd.concat([df_atual,df6]).drop_duplicates(ignore_index=True)
            df = df.drop_duplicates(['titulo','empresa','fonte', 'texto_completo'], keep='first')
            df.to_excel(base_noticias, index=False)






2024-02-22 22:17:36.264210 Buscando empresa brasil biofuels
...... buscando no google brasil+biofuels
2024-02-22 22:18:38.283955 ... noticias encontradas: 316
2024-02-22 22:18:38.286377 ... noticias novas: 256
